In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [3]:
y_preds = {}

In [4]:
# prepare training data

In [9]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=20, sample_weights_size=20, is_oversample=False, include_new_data=False)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","sentiment","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, discretize_size=50,
                                    dis_strategy="quantile",normalize=True)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=5000)

feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%Data size: 3926

Finished windowfying
Featurizing calculate_feats=True, normalize=True, discretize=True, discretize_size=50, include_feats=['first_prons', 'sentiment', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 3926, 3926
Data size: 4650, 4650
Calculating first prons
Calculating sentiment
Calculating NSSI words
Calculating first prons
Calculating sentiment
Calculating NSSI words
Normalizing features
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 5 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.loc

Is the combined the same from tfidf: False


In [10]:
# traditional classifiers

In [11]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.81      0.99      0.89      3602
           1       0.89      0.20      0.33      1048

    accuracy                           0.81      4650
   macro avg       0.85      0.60      0.61      4650
weighted avg       0.83      0.81      0.77      4650

[[3577   25]
 [ 838  210]]
{'precision': 0.8032786885245902, 'recall': 0.47115384615384615, 'F1': 0.593939393939394, 'ERDE_5': 0.2526829310647792, 'ERDE_50': 0.13699847425515232, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.5684750136564214}


In [12]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.79      0.99      0.88      3602
           1       0.79      0.12      0.21      1048

    accuracy                           0.79      4650
   macro avg       0.79      0.56      0.55      4650
weighted avg       0.79      0.79      0.73      4650

[[3567   35]
 [ 920  128]]
{'precision': 0.7708333333333334, 'recall': 0.3557692307692308, 'F1': 0.4868421052631579, 'ERDE_5': 0.25215380161488077, 'ERDE_50': 0.16478603244862924, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.4659693821660113}


In [13]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 3

In [14]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 3926
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00043: early stopping
Evaluating
146/146 [==============================] - 2s 9ms/step - loss: 0.7576 - tp: 2.0000 - fp: 1.0000 - tn: 3601.0000 - fn: 1046.0000 - accuracy: 0.7748 - precision: 0.6667 - recall: 0.0019 - f1_metric: 0.0020
Test Score: 0.7576147317886353
Test Accuracy: 2.0
146/146 [==============================] - 1s 8ms/step
Entered here
              precision    recall  f1-score   support

           0       0.77      1.00      0.87      3602
           1       0.67      0.00      0.00      1048

    accuracy                           0.77      4650
   macro avg       0.72      0.50      0.44      4650
weighted avg       0.75      0.77      0.68      4650

[[3601    1]
 [1046    2]]
Finished training and evaluation
{'precision': 0.6666666666666666, 'recall': 0.019230769230

/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib64/python3.6/site

Data size: 3926
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00031: early stopping
Evaluating
146/146 [==============================] - 2s 9ms/step - loss: 0.5559 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 3602.0000 - fn: 1048.0000 - accuracy: 0.7746 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_metric: 0.0000e+00
Test Score: 0.5558797717094421
Test Accuracy: 0.0
146/146 [==============================] - 2s 7ms/step
Entered here
              precision    recall  f1-score   support

           0       0.77      1.00      0.87      3602
           1       0.00      0.00      0.00      1048

    accuracy                           0.77      4650
   macro avg       0.39      0.50      0.44      4650
weighted avg       0.60      0.77      0.68      4650

[[3602    0]
 [1048    0]]
Finished training and evaluation
{'precision': 0, 'recall': 0, 'F1': 0, 'ERDE_5': 0.2458628841607565, 'ERDE_50': 0.2458628841607565, 'median_latency_tps': nan, 'median_pe

/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib64/python3.6/site

In [15]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [16]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred



Starting training with cnn_model=lstm_model_32 and maxlen=1000 and batch size=32
Generating embeddings
Data size: 3926
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00076: early stopping
Evaluating
146/146 [==============================] - 24s 150ms/step - loss: 0.6602 - tp: 419.0000 - fp: 258.0000 - tn: 3344.0000 - fn: 629.0000 - accuracy: 0.8092 - precision: 0.6189 - recall: 0.3998 - f1_metric: 0.2838
Test Score: 0.6601938009262085
Test Accuracy: 419.0
146/146 [==============================] - 23s 151ms/step
Entered here
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      3602
           1       0.62      0.40      0.49      1048

    accuracy                           0.81      4650
   macro avg       0.73      0.66      0.68      4650
weighted avg       0.79      0.81      0.79      4650

[[3344  258]
 [ 629  419]]
Finished training and evaluation
{'precision': 0.4785276073619632, 'recall':

TypeError: '>' not supported between instances of 'dict_keys' and 'int'

In [21]:
if iterations > 0:
    y_preds[model_name] = model_resuls[max(model_resuls.keys())]

In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [22]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["cnn_model"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

{'precision': 0.8648648648648649, 'recall': 0.3076923076923077, 'F1': 0.45390070921985815, 'ERDE_5': 0.2486934583548893, 'ERDE_50': 0.17311894662128827, 'median_latency_tps': 12.5, 'median_penalty_tps': 0.044819781830275796, 'speed': 0.9551802181697242, 'latency_weighted_f1': 0.4335569784600167}


{'precision': 0.8648648648648649,
 'recall': 0.3076923076923077,
 'F1': 0.45390070921985815,
 'ERDE_5': 0.2486934583548893,
 'ERDE_50': 0.17311894662128827,
 'median_latency_tps': 12.5,
 'median_penalty_tps': 0.044819781830275796,
 'speed': 0.9551802181697242,
 'latency_weighted_f1': 0.4335569784600167}

In [23]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

{'precision': 0.8490566037735849, 'recall': 0.4326923076923077, 'F1': 0.5732484076433121, 'ERDE_5': 0.2503793273960633, 'ERDE_50': 0.14412979449949517, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.5486711265304729}


{'precision': 0.8490566037735849,
 'recall': 0.4326923076923077,
 'F1': 0.5732484076433121,
 'ERDE_5': 0.2503793273960633,
 'ERDE_50': 0.14412979449949517,
 'median_latency_tps': 12.0,
 'median_penalty_tps': 0.042873701496841665,
 'speed': 0.9571262985031583,
 'latency_weighted_f1': 0.5486711265304729}

In [24]:
y_pred = ensemble_vote(np.array([y_preds["bayes"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

{'precision': 0.8, 'recall': 0.3076923076923077, 'F1': 0.4444444444444444, 'ERDE_5': 0.2504266322370938, 'ERDE_50': 0.17486265501959317, 'median_latency_tps': 12.5, 'median_penalty_tps': 0.044819781830275796, 'speed': 0.9551802181697242, 'latency_weighted_f1': 0.4245245414087663}


{'precision': 0.8,
 'recall': 0.3076923076923077,
 'F1': 0.4444444444444444,
 'ERDE_5': 0.2504266322370938,
 'ERDE_50': 0.17486265501959317,
 'median_latency_tps': 12.5,
 'median_penalty_tps': 0.044819781830275796,
 'speed': 0.9551802181697242,
 'latency_weighted_f1': 0.4245245414087663}

In [25]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

{'precision': 0.8135593220338984, 'recall': 0.46153846153846156, 'F1': 0.588957055214724, 'ERDE_5': 0.25210327854603726, 'ERDE_50': 0.1387813043162381, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.563706286234989}


{'precision': 0.8135593220338984,
 'recall': 0.46153846153846156,
 'F1': 0.588957055214724,
 'ERDE_5': 0.25210327854603726,
 'ERDE_50': 0.1387813043162381,
 'median_latency_tps': 12.0,
 'median_penalty_tps': 0.042873701496841665,
 'speed': 0.9571262985031583,
 'latency_weighted_f1': 0.563706286234989}